In [2]:
import pandas as pd
import numpy as np

# Define the file path for the dataset
file_path = r'D:\MLOPS\MLOPS Project\energy_consumption_prediction\data\raw\household_power_consumption.txt'

# Load the dataset
# The dataset is semicolon-separated and missing values are represented by '?'
df = pd.read_csv(file_path, sep=';', low_memory=False, na_values=['?'])

# Display the first few rows and the data types of the columns
print("Original Data Head:")
print(df.head())
print("\nOriginal Data Types:")
print(df.info())

Original Data Head:
         Date      Time  Global_active_power  Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00                4.216                  0.418   234.84   
1  16/12/2006  17:25:00                5.360                  0.436   233.63   
2  16/12/2006  17:26:00                5.374                  0.498   233.29   
3  16/12/2006  17:27:00                5.388                  0.502   233.74   
4  16/12/2006  17:28:00                3.666                  0.528   235.68   

   Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
0              18.4             0.0             1.0            17.0  
1              23.0             0.0             1.0            16.0  
2              23.0             0.0             2.0            17.0  
3              23.0             0.0             1.0            17.0  
4              15.8             0.0             1.0            17.0  

Original Data Types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259

In [3]:
# Combine 'Date' and 'Time' into a single datetime column
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True)

# Set the new 'datetime' column as the index
df.set_index('datetime', inplace=True)

# Drop the original 'Date' and 'Time' columns
df.drop(['Date', 'Time'], axis=1, inplace=True)

# Display the first few rows with the new index
print("\nData with Datetime Index:")
print(df.head())
print("\nUpdated Data Types:")
print(df.info())


# Check for missing values in each column
print("\nMissing Values Before Handling:")
print(df.isnull().sum())

# Handle missing values using forward fill
df.ffill(inplace=True)

# Verify that there are no more missing values
print("\nMissing Values After Handling:")
print(df.isnull().sum())


Data with Datetime Index:
                     Global_active_power  Global_reactive_power  Voltage  \
datetime                                                                   
2006-12-16 17:24:00                4.216                  0.418   234.84   
2006-12-16 17:25:00                5.360                  0.436   233.63   
2006-12-16 17:26:00                5.374                  0.498   233.29   
2006-12-16 17:27:00                5.388                  0.502   233.74   
2006-12-16 17:28:00                3.666                  0.528   235.68   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
datetime                                                                
2006-12-16 17:24:00              18.4             0.0             1.0   
2006-12-16 17:25:00              23.0             0.0             1.0   
2006-12-16 17:26:00              23.0             0.0             2.0   
2006-12-16 17:27:00              23.0             0.0             1.0   
20

In [4]:
# Calculate the remainder of the sub-metering
values = df.values.astype('float32')
df['Sub_metering_4'] = (values[:, 0] * 1000 / 60) - (values[:, 4] + values[:, 5] + values[:, 6])

print("\nData with Sub_metering_4:")
print(df.head())


Data with Sub_metering_4:
                     Global_active_power  Global_reactive_power  Voltage  \
datetime                                                                   
2006-12-16 17:24:00                4.216                  0.418   234.84   
2006-12-16 17:25:00                5.360                  0.436   233.63   
2006-12-16 17:26:00                5.374                  0.498   233.29   
2006-12-16 17:27:00                5.388                  0.502   233.74   
2006-12-16 17:28:00                3.666                  0.528   235.68   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
datetime                                                                
2006-12-16 17:24:00              18.4             0.0             1.0   
2006-12-16 17:25:00              23.0             0.0             1.0   
2006-12-16 17:26:00              23.0             0.0             2.0   
2006-12-16 17:27:00              23.0             0.0             1.0   
20

In [6]:
# Resample the data to an hourly frequency, taking the mean of the values
df_hourly = df.resample('H').mean()

print("\nHourly Resampled Data Head:")
print(df_hourly.head())
print("\nShape of the hourly resampled data:", df_hourly.shape)


Hourly Resampled Data Head:
                     Global_active_power  Global_reactive_power     Voltage  \
datetime                                                                      
2006-12-16 17:00:00             4.222889               0.229000  234.643889   
2006-12-16 18:00:00             3.632200               0.080033  234.580167   
2006-12-16 19:00:00             3.400233               0.085233  233.232500   
2006-12-16 20:00:00             3.268567               0.075100  234.071500   
2006-12-16 21:00:00             3.056467               0.076667  237.158667   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
datetime                                                                
2006-12-16 17:00:00         18.100000             0.0        0.527778   
2006-12-16 18:00:00         15.600000             0.0        6.716667   
2006-12-16 19:00:00         14.503333             0.0        1.433333   
2006-12-16 20:00:00         13.916667             0.

C:\Users\khush\AppData\Local\Temp\ipykernel_19048\3914638576.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.resample('H').mean()


In [7]:
# Save the cleaned and resampled dataframe to a new CSV file
df_hourly.to_csv('household_power_consumption_hourly.csv')

print("\nPreprocessed hourly data saved to 'household_power_consumption_hourly.csv'")


Preprocessed hourly data saved to 'household_power_consumption_hourly.csv'
